In [78]:
import pandas as pd

In [82]:
df = pd.read_csv('train1.csv')

In [83]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      135 non-null    int64 
 1   title   132 non-null    object
 2   author  122 non-null    object
 3   text    135 non-null    object
 4   label   135 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 5.4+ KB


In [85]:
df.describe()

,id,label
count,135.000000,135.000000
mean,67.000000,0.437037
std,39.115214,0.497867
min,0.000000,0.000000
25%,33.500000,0.000000
50%,67.000000,0.000000
75%,100.500000,1.000000
max,134.000000,1.000000


In [86]:
df.isnull().sum()

id         0
title      3
author    13
text       0
label      0
dtype: int64

In [87]:
df = df.fillna('')
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [88]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [89]:
df = df.drop(['id', 'title', 'author'], axis=1)

In [90]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \r\nAn Iranian woman has been sentenced ...,1


In [91]:
from nltk.corpus import stopwords

In [92]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

In [93]:
import re


In [94]:
port_stem = PorterStemmer()

In [95]:
port_stem

<PorterStemmer>

In [96]:
port_stem.stem('Hi this is chando')

'hi this is chando'

In [97]:
def stemming(content):
    con = re.sub('[^a-zA-Z]', ' ', content)
    con = con.lower()
    con = con.split()
    con = [port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con = ' '.join(con)
    return con

In [98]:
stemming('Hi this is chando')

'hi chando'

In [99]:
df['text'] = df['text'].apply(stemming)

In [100]:
x = df['text']
y = df['label']

In [101]:
y.shape

(135,)

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [105]:
vect=TfidfVectorizer()

In [106]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [107]:
x_test.shape

(27, 7890)

In [108]:
from sklearn.tree import DecisionTreeClassifier

In [109]:
model=DecisionTreeClassifier()

In [110]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [111]:
prediction=model.predict(x_test)

In [112]:
prediction

array([1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0], dtype=int64)

In [113]:
model.score(x_test, y_test)

0.7037037037037037

In [114]:
import pickle

In [115]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [116]:
pickle.dump(model, open('model.pkl', 'wb'))

In [117]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [118]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [119]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [120]:
val=fake_news("""It Literally Hurts My Brain to Read the Economic Idiocy Emitted by Trumpkins | Libertarian,beforeitsnews.com,"(Before It's News)
(Don Boudreaux)
Tweet
Here’s a letter to the Wall Street Journal :
Wilbur Ross’s and Peter Navarro’s defense of Donald Trump’s economic policies is mostly a mash of bunkum (“ A Vote for Trump Is a Vote for Growth ,” Oct. 26). Consider this claim: “Donald Trump will cut taxes, reduce regulation … and eliminate our trade deficit through muscular trade negotiations that increase exports, [and] reduce imports….”
Cut taxes? Bunk. Trump famously promises to raise taxes on Americans who buy imports. Reduce regulation? Rubbish. Trump promises more government intrusions into Americans’ commerce with foreigners.
As for ‘eliminating’ our trade deficit, Trump might indeed succeed on that front. But such ‘success’ would be regrettable, for it would be the inevitable outcome of the American economy being made an unattractive destination for investment. (Ross and Navarro seem to be unaware that to “eliminate our trade deficit” – such as was done, for example, during the Great Depression – is to eliminate net contributions by foreigners to increasing the size of America’s capital stock.)
But Trump’s most absurd promise is to enrich Americans by increasing exports and reducing imports. Imports are what we voluntarily buy and exports are the price we pay. Therefore, a policy meant to increase exports while decreasing imports is a policy meant to force Americans to pay more to foreigners and to receive less in return – a decidedly unartful deal the architect of which would deserved to be fired.
But the Trump camp’s confusion runs even more deeply. Exporting for Americans is worthwhile only because it supplies us with the means to purchase imports, either currently or in the future. So a policy that aims both to increase exports and to decrease imports is akin to a policy that aims both to increase people’s spending power and to decrease it. It’s a policy meant to give Americans greater means for acquiring imports as it simultaneously strips Americans of the freedom to use those means. It’s the economic policy equivalent of an attempt to square a circle.
Sincerely,
Donald J. Boudreaux
Professor of Economics
and
Martha and Nelson Getchell Chair for the Study of Free Market Capitalism at the Mercatus Center
George Mason University
Fairfax, VA 22030""")

In [121]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

unreliable
